# 1. Практическое задание. Обучение полносвязной нейронной сети.

**ФИО**: ВАШЕ ФИО

### Большая просьба

Называйте файл hw1_Фамилия.ipynb

In [1]:
import numpy as np
import torch
import cv2

from glob import glob
from collections import OrderedDict
from matplotlib import pyplot as plt

from torch import nn
from torch.autograd import Function
from torch.autograd import gradcheck
from torch.optim import Optimizer, Adam
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision import transforms

## 1. Загрузка данных (2 балла)

Если вам требуется работать с каким-нибубь набором данных (dataset), то прежде всего проверьте нет ли его среди встроенных наборов данных https://pytorch.org/vision/stable/datasets.html.

В текущем домашнем задании мы будем работать с набором данных FashionMNIST. Это не совсем про тексты, но зато он численный и его можно вытянуть в вектор. Это легко и полезно, чтобы отработать разные приемы. Он присутствует в списке встроенных наборов данных, однако мы воспользуемся реализацией только для удобного и быстрого способа скачать наборы данных. Ниже предлагается реализовать собственный класс для считывания, обработки и упаковки данных.

In [ ]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True
)

Воспользуемся функцией загрузки данных из репозитория наборов данных.

In [ ]:
! ls data/FashionMNIST/raw

In [4]:
#https://github.com/zalandoresearch/fashion-mnist/blob/master/utils/mnist_reader.py

def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

Для удобства PyTorch предоставляет ряд базовых классов `Dataset, DataLoader`, от которых предлагается отнаследоваться при разработке пользовательских классов. Базовый класс `Dataset` используется для загрузки и обработки данных, класс `DataLoader` используется для управления процессом загрузки данных, позволяет в многопоточном режиме загружать данные и упаковывать их.
Эти вспомогательные классы находятся в модуле `torch.utils.data`. 

При наследовании от класса `torch.utils.data.Dataset` требуется переопределить метод `__len__`, который возвращает количество примеров в наборе данных, а также метод `__getitem__`, который позволяет получить доступ к примеру из набора данных по индексу.

Реализуем класс для FasionMnist.

Элементами датасета должны являться пары '(np.array, int)', массив имеет размерность `(28, 28)`, тип элемента `np.float32`.

In [ ]:
import os

class FashionMnist(Dataset):
    def __init__(self, path, train=True, image_transform=None, 
                 label_transform=None):

        ### YOUR CODE HERE
        ### LOAD IMAGES AND LABELS WITH FUNCTION
        ...


        ### ALSO PROVIDE TRANSFORMS TO APPLY
        self.image_transform = 
        self.label_transform = 


    def __len__(self,):
        
        ### YOUR CODE
        # RETURN LENGTH OF DATASET


    def __getitem__(self, idx):
        

        ### YOUR CODE HERE
        ### APPLY TRANSFORMS AND RETURN ELEMENTS



In [6]:
test_dataset = FashionMnist("data/FashionMNIST", train=False)
train_dataset = FashionMnist("data/FashionMNIST")

Визуализируйте случайные элементы набора данных.

In [ ]:
### YOUR CODE HERE

В конструктор `Dataset` можно передать объект `torchvision.transforms`, который позволяет преобразовать исходные данные. Преобразование `torchvision.transforms.ToTensor` позволяет преобразоать данные из типа `PIL Image` и `numpy.float32` в тип `torch.float32`

Реализуйте собственную поддержку преобразований в `FashionMnist`. Проверьте, что приведение типов работает корректно. 

In [8]:
class ToTensor:
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        ### YOUR CODE HERE


# SHOULD BE CALLABLE ToTensor(x)

In [9]:
transform = ToTensor()

### YOUR CODE HERE
# init dataset with your transform and check datatype


Элементы набора данных могут быть объединены в пакеты (batch) явно и неявно. Если данные могут быть сконкатенированы или обЪединены каким-нибудь тривиальным способом, то можно не передавать никаких дополнительных парамертов в `torch.utils.data.Dataloader`.

In [11]:
test_dataloader = DataLoader(test_dataset, batch_size=15, num_workers=2, shuffle=True)
batch = next(iter(test_dataloader))

In [ ]:
print(f"The length of the batch is {len(batch)}")
print(f"The shape of the batch[0] is {batch[0].shape}")

Однако, если наша структура данных не позволяет нам использовать объединение по умолчанию, то можно написать собственную функцию, которая будет пакетировать данные. 

Реализуйте функцию, преобразующую последовательность элементов массива в пакет (batch).

In [13]:
### YOUR CODE HERE
### WRITE A COLLATE FUNCTION and use it with dataloaders

Убедитесть, что все работает корректно. 

In [14]:
test_dataloader = ### YOUR CODE HERE
train_dataloader = ### YOUR CODE HERE
batch = next(iter(test_dataloader))

In [ ]:
print(f"The length of the batch is {len(batch)}")
print(f"The shape of the batch[0] is {batch[0].shape}")

## 2. Реализация модулей нейронной сети (3 балла)

В этом разделе мы полностью реализуем модули для полносвязанной сети. 

Для начала нам понадобится реализовать прямой и обратный проход через слои. 

Наши слои будут соответствовать следующему интерфейсу (на примере "тождественного" слоя):

Сначала, мы реализуем функцию и её градиент. 

In [7]:
class IdentityFunction(Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """
    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        return input

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        return grad_output

Разработанную функцию обернем классом `IdentityLayer`, все слои в `PyTorch` должны быть наследниками базового класса `nn.Module()`


In [8]:
class IdentityLayer(nn.Module):
    def __init__(self):
        # An identity layer does nothing
        super().__init__()
        self.identity = IdentityFunction.apply
    
    def forward(self, inp):
        # An identity layer just returns whatever it gets as input.
        return self.identity(inp)


### 2.1 Функция активации ReLU
Для начала реализуем функцию активации, слой нелинейности `ReLU(x) = max(x, 0)`. Параметров у слоя нет. Метод `forward` должен вернуть результат поэлементного применения `ReLU` к входному массиву, метод `backward` - градиент функции потерь по входу слоя. В нуле будем считать производную равной 0. Обратите внимание, что при обратном проходе могут понадобиться величины, посчитанные во время прямого прохода, поэтому их стоит сохранить в `ctx`.

In [18]:
class ReLUFunction(Function):
    @staticmethod
    def forward(ctx, input):
                
        ### YOUR CODE HERE
        ### CALCULATE RELU FUNCTION WITH TORCH AND SAVE SOMETHING FOR BACKWARD VIA CTX

    @staticmethod
    def backward(ctx, grad_output):
    
        ### YOUR CODE HERE
        ### GET SOMETHING FROM FORWARD AND CALCULATE GRADIENT
        # CHECK LECTIONS OR GOOGLE
        out = ... 

        return grad_output * out


In [19]:
class ReLU(nn.Module):
    def __init__(self):
        
        super().__init__()
        self.relu = ReLUFunction.apply

    def forward(self, input):

        return self.relu(input)


Не забываем после реализации функции проверить градиент, испльзуя функцию `gradcheck`.

In [20]:
torch.manual_seed(0)

x = torch.rand((7,15), requires_grad = True, dtype=torch.double)
relu = ReLU()

assert gradcheck(relu, x)

In [21]:
torch_relu = torch.relu
our_relu = ReLU()

assert torch.norm(torch_relu(x.float()) - our_relu(x)) < 1e-5

### 2.2 Линейный слой (linear, fully-connected)
Далее реализуем полносвязный слой без нелинейности. У слоя два набора параметра: матрица весов (weights) и вектор смещения (bias).

In [22]:
class LinearFunction(Function):
    @staticmethod
    def forward(ctx, inp, weight, bias):

        ### YOUR CODE HERE
        ### CALCULATE OUTPUT 
        ### AND SAVE SOMETHING FOR BACKWARD

        return output

    @staticmethod
    def backward(ctx, grad_output):

        # GET SOMETHING FROM BACKWARD

        # CHECK HOW BACKWARD PERFORMED
        grad_bias = grad_output.sum(0)
        grad_weight = grad_output.T @ inp
        grad_input =  grad_output @ weight 


        return grad_input, grad_weight, grad_bias

In [23]:
class Linear(nn.Module):
    def __init__(self, input_units, output_units):
        super().__init__()

        ### YOUR CODE HERE
        ### initialize weights and bias with small random numbers or xavier
        ### do not forget to make them torch.nn.Parameter 

        self.linear = LinearFunction.apply
        
    def forward(self,inp):
        
        return self.linear(inp, self.weight, self.bias)


Проверим градиент, а также сравним с работой нашего модуля с имплементированным в `PyTorch`. 

Проверка градиента:

In [24]:
torch.manual_seed(0)

x = torch.rand((6,12), requires_grad = True, dtype=torch.double)
linear = Linear(12, 14)

assert gradcheck(linear, x)

Сравнение с `PyTorch`. 

In [ ]:
output_units = 32
input_units = 15

x = torch.rand((16,15), requires_grad = True, dtype=torch.double)


weight = torch.rand(size=(output_units, input_units), dtype=torch.double)
bias = torch.rand(size=(output_units,), dtype=torch.double)

torch_linear = torch.nn.Linear(input_units, output_units, dtype=torch.double)
our_linear = Linear(input_units, output_units)


state_dict = OrderedDict([("weight", weight), ("bias", bias)])
torch_linear.load_state_dict(state_dict)
our_linear.load_state_dict(state_dict)



torch_forward = torch_linear.forward(x)
our_forward = our_linear(x)
assert torch.allclose(torch_forward, our_forward)

## 3. Сборка и обучение нейронной сети (5 баллов)

Реализуйте произвольную нейросеть, состоящую из ваших блоков. Она должна состоять из нескольких полносвязанных слоев.

In [33]:
class Network(nn.Module):
    def __init__(self, input_size=28*28, hidden_layers_size=32, num_layers=5,
                 num_classes=10):
        super().__init__()
        

        ### YOUR CODE HERE
        ### STACK LAYERS WITH DEFINED PARAMETERS
        ### USE nn.Dropout, your linear, your relu and whatever you like
        ### LAST LAYER SHOULD BE nn.LogSoftmax
    def forward(self, inp):
        
        ### YOUR CODE HERE
        ### APPLY YOUR NET TO THE INPUT


Ниже вам предстоит написать цикл обучения. Сначала это бывает больно, но потом уже на автомате делается, в начале так всегда


In [34]:
class EmptyContext:
    def __enter__(self):
        pass
    
    def __exit__(self, *args):
        pass

In [35]:
# accuract metric for our classififcation
def accuracy(model_labels, labels):
  return torch.mean((model_labels == labels).float())

In [36]:
def perform_epoch(model, loader, criterion, 
                optimizer=None, device=None):
    is_train = optimizer is not None
    ### YOUR CODE HERE
    ### MOVE MODEL TO DEVICE
    ### CHANGE MODEL TO TRAIN OR EVAL MODE

    ### SET LOGGING VALUES
    ### ITERATE OVER DATALOADER
    ### MOVE BATCH AND LABELS TO DEVICE
    ### GET MODEL OUTPUT
    ### GET MODEL PREDICTIONS (from the probabilites)
    ### CALCULATE LOSS
    ### BACKWARD IF TRAIN 
    ### STEP WITH OPTIMIZER (DONT FORGET TO ZERO GRAD)
    ### UPDATE LOGGING VALUES WITH LOSS AND ACCURACY
    ### RETURN LOGGED VALUES

Теперь обучим нашу нейронную сеть.

In [ ]:
# INIT YOUR MODEL
# CRITERION
#  AND OPTIMIZER
# Add device

# SET NUMBER OF EPOCHS
# ITERATE OVER NUMBERS OF EPOCH
# TRAIN AND VALIDATE
# LOG METRICS FOR TRAIN AND VAL LIKE BELOW. (YOU MAY USE YOUR OWN WAY)
print(f"Epoch - {epoch} : loss {loss}, accuracy {acc}")


Дальше:
- Проведите эксперименты с числом слоев. 
- Постройте графики зависимости качества модели на тренировочной и тестовой выборках от числа слоев. Для получения статистически значимых результатов повторите эксперименты несколько раз.
- Сделайте выводы. 

## 4. Бонусная часть.

### 4.1 Реализация метода оптимизации (1 балл).
Реализуйте сами метод оптимизации  для рассмотренной выше архитектуры. Вы можете выбрать произвольный метод от градиентного спуска до современных вариантов. Продемонстрируйте правильную работу метода оптимизации, сравните его работы с Adam. 

**Дополнительные баллы** вы получите, если метод будет уникален среди сдавших задание. 

In [ ]:
class SotaOptimizer(Optimizer):
    def __init__(self, params, lr=1e-3):
        defaults = dict(lr=lr)
        super(SotaOptimizer, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(SotaOptimizer, self).__setstate__(state)

    @torch.no_grad()
    def step(self,):

        for group in self.param_groups:
            lr = group['lr']
            for p in group['params']:
                if p.grad is not None:
                    p.data.add_(-lr*p.grad)

### 4.2 Реализация современной функции активации (1 балл).
Реализуйте одну из современных функций активации. Например, `Hardswish` или `GELU`. Сравните сеть с вашей активацией и с `ReLU`. 

**Дополнительные баллы** вы получите, если функция будет уникальна среди сдавших задание. 